
<center><h1><i> "R to Python" <i></h1></center>

### What does the R code even do?

    To be able to translate something we should first understand what it's doing in the first place, we copy and pasted the R code into chatgpt with the prompt "Explain what this R code is doing" so we could get a better understand of what we're dealing with.

### ChatGPT Response Summarized w/ Translation:


Data Preparation:

- It first loads a variety of libraries (like dplyr, lubridate, data.table, factoextra, etc.) to handle data manipulation, date conversion, plotting, and clustering.
    The dataset is copied into a variable named “tidy” and its columns are renamed to more meaningful labels (e.g. “pnr” for patient number, “eksd” for event date, “ATC” for drug classification, etc.).

- The date column “eksd” is converted into a proper date format using the mdy() function from the lubridate package.

In [2]:
import pandas as pd
import matplotlib as plt

data = pd.read_csv("../Dataset/ExamplePats.csv")

data.head()


,pnr,eksd,perday,ATC,dur_original
0,1,2033-04-26,4,medA,50
1,1,2033-07-04,4,medB,30
2,1,2033-08-03,4,medB,30
3,1,2033-08-17,4,medB,30
4,1,2033-10-13,4,medB,30


- In this cell we load in the csv into our data variable and we call the data.head() function to see if we've successfully imported our data.
  
- Since the dataset has been properly formatted before hand, we wouldn't to change any formatting regarding the columns of the data and the format of the dates.

Function ‘See’: Analyzing Prescription Intervals for a Specific Drug
The function See(arg1) takes a drug code (e.g., "medA" or "medB") and performs the following steps:

- Subsetting the Data:
It filters the dataset to include only records where the “ATC” column matches the given drug code.

- Calculating Time Intervals:
For each patient (grouped by “pnr”), the function computes the time difference (event interval) between consecutive prescription dates (“eksd”). It does this by arranging the events chronologically and using a lag function.

- Random Sampling:
For each patient, one prescription interval is randomly selected. This helps to standardize the sample for further analysis.

- ECDF Plotting:
An empirical cumulative distribution function (ECDF) is generated for the event intervals. The function then creates two plots: one for the lower 80% of the ECDF (to focus on the bulk of the data, likely reducing the influence of outliers) and one for the full ECDF. Both are saved as PNG files.

- Histogram and Density Plot:
A histogram of the number of events per patient is plotted. Then, the logarithm of the event intervals is taken and its density is computed and plotted. This log transformation can help normalize skewed interval data.

- Clustering Analysis:
The code performs silhouette analysis to determine the optimal number of clusters for the log-transformed intervals. Using k-means clustering, it then groups the intervals into clusters. For each cluster, summary statistics (minimum, maximum, median) are computed.
The function then assigns a cluster (and the corresponding median duration) to each event based on whether its interval falls within the range of that cluster.

- Merging Results:
Finally, the original data (for the specified drug) is merged with the cluster results so that each record now includes an assigned “Median” (typical duration) and “Cluster” identifier.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Helper function to compute ECDF
def ecdf(data):
    x = np.sort(data)
    y = np.arange(1, len(x)+1) / len(x)
    return x, y

def see(drug_code, data):
    # 1. Subset the data by the given drug code
    drug_data = data[data['ATC'] == drug_code].copy()
    
    # 2. Convert the prescription date column to datetime and sort by patient and date
    drug_data['eksd'] = pd.to_datetime(drug_data['eksd'])
    drug_data = drug_data.sort_values(by=['pnr', 'eksd'])
    
    # Calculate previous prescription date for each patient using groupby and shift
    drug_data['prev_eksd'] = drug_data.groupby('pnr')['eksd'].shift(1)
    
    # Remove the first prescription for each patient (no previous date)
    drug_data = drug_data.dropna(subset=['prev_eksd']).copy()
    
    # 3. Calculate the time interval (in days) between consecutive prescriptions
    drug_data['event_interval'] = (drug_data['eksd'] - drug_data['prev_eksd']).dt.days
    
    # 4. For each patient, randomly select one prescription interval
    sampled = drug_data.groupby('pnr').apply(lambda x: x.sample(1)).reset_index(drop=True)
    
    # 5. ECDF Plotting
    # Full ECDF
    x_ecdf, y_ecdf = ecdf(sampled['event_interval'])
    plt.figure()
    plt.plot(x_ecdf, y_ecdf, marker='.', linestyle='none')
    plt.xlabel('Event Interval (days)')
    plt.ylabel('ECDF')
    plt.title(f'Full ECDF for {drug_code}')
    plt.savefig(f'ECDF_{drug_code}_full.png')
    plt.close()
    
    # ECDF for lower 80% of intervals
    cutoff = np.percentile(sampled['event_interval'], 80)
    mask = sampled['event_interval'] <= cutoff
    x_ecdf_80, y_ecdf_80 = ecdf(sampled.loc[mask, 'event_interval'])
    plt.figure()
    plt.plot(x_ecdf_80, y_ecdf_80, marker='.', linestyle='none')
    plt.xlabel('Event Interval (days)')
    plt.ylabel('ECDF')
    plt.title(f'ECDF up to 80th percentile for {drug_code}')
    plt.savefig(f'ECDF_{drug_code}_80.png')
    plt.close()
    
    # 6. Histogram of the number of events per patient
    events_per_patient = drug_data.groupby('pnr').size()
    plt.figure()
    events_per_patient.plot(kind='bar')
    plt.xlabel('Patient (pnr)')
    plt.ylabel('Number of events')
    plt.title(f'Prescription Events per Patient for {drug_code}')
    plt.savefig(f'Histogram_{drug_code}.png')
    plt.close()
    
    # Density plot of log-transformed event intervals (using only intervals <= cutoff)
    filtered_intervals = sampled[sampled['event_interval'] <= cutoff]['event_interval']
    log_intervals = np.log(filtered_intervals)
    plt.figure()
    sns.kdeplot(log_intervals, shade=True)
    plt.xlabel('log(Event Interval)')
    plt.title(f'Density Plot of log(Event Interval) for {drug_code}')
    plt.savefig(f'Density_{drug_code}.png')
    plt.close()
    
    # 7. Clustering Analysis on log-transformed intervals
    X = log_intervals.values.reshape(-1, 1)
    best_k = 2
    best_score = -1
    scores = {}
    # Try cluster numbers from 2 to 5
    for k in range(2, 6):
        kmeans = KMeans(n_clusters=k, random_state=1234).fit(X)
        score = silhouette_score(X, kmeans.labels_)
        scores[k] = score
        if score > best_score:
            best_score = score
            best_k = k
    print("Silhouette scores for different k:", scores)
    print("Optimal number of clusters:", best_k)
    
    # Apply k-means clustering with the optimal number of clusters
    kmeans = KMeans(n_clusters=best_k, random_state=1234).fit(X)
    sampled['cluster'] = kmeans.labels_
    
    # Compute summary statistics for each cluster (on the original event_interval)
    cluster_stats = sampled.groupby('cluster')['event_interval'].agg(['min', 'max', 'median']).reset_index()
    
    # 8. Merge cluster statistics with the sampled data
    sampled = pd.merge(sampled, cluster_stats, on='cluster', suffixes=('', '_cluster'))
    
    # Merge the cluster info back into the full drug_data on patient ID (pnr)
    merged = pd.merge(drug_data, sampled[['pnr', 'median', 'cluster']], on='pnr', how='left')
    
    # If any patient did not receive a cluster assignment, fill with overall median
    overall_median = sampled['median'].median()
    merged['median'] = merged['median'].fillna(overall_median)
    merged['cluster'] = merged['cluster'].fillna(-1)
    
    return merged

# Example usage:
# Assuming you have your DataFrame loaded as "tidy"
# result_medA = see("medA", tidy)
# result_medB = see("medB", tidy)
